In [ ]:
import xarray as xr
import numpy as np
import dask
import dask.array as da
from dask.diagnostics import ProgressBar

In [ ]:
from linearsim import time_domain_ras

# Generate a dataset of input spectra 

In [ ]:
ds_spec = xr.Dataset()
ds_spec['Gamma'] = xr.DataArray([3.3],dims='gamma')
ds_spec['Tp'] = xr.DataArray([12.],dims='tp')
ds_spec['Duration'] = xr.DataArray(da.array([2,5,10,20,40,60])*60,dims='duration')
n_seed = 1000
ds_spec['Seed'] = xr.DataArray(da.arange(n_seed,chunks=(200,)).astype('int'),dims='seed')
ds_spec['this_seed'] = xr.DataArray(da.random.choice(int(1E9), size=(len(ds_spec['Tp']), len(ds_spec['Gamma']), len(ds_spec['Duration']), n_seed)).astype('int'), dims=('tp','gamma','duration','seed'))
ds_spec = ds_spec.chunk({'tp':1,'gamma':-1,'duration':1,'seed':200})
ds_spec['hs'] = 1.0
ds_spec['dt'] = 0.5
ds_spec['fft_min_duration']=60*60

def set_coords(ds):
    ds = ds.rename({'Duration':'duration','Gamma':'gamma','Seed':'seed','Tp':'tp'})
    ds = ds.assign_coords({'duration':ds['duration'],'gamma':ds['gamma'],'seed':range(len(ds['seed'])),'tp':ds['tp']})
    ds = ds.set_coords(['hs','tp','dt','fft_min_duration'])
    return ds

ds_spec = set_coords(ds_spec)

ds_spec

In [ ]:
# Names of output variables returned from time_domain_ras function
output_variables = 'Tz, Tm01, Hm0, Hs, H13, H13_unbiased, Hmax, HmaxT, Cmax, CmaxT2, r_spectra, r_sample, r_unbiased, k3, k4, seed'
outputs = xr.apply_ufunc(time_domain_ras,
                        ds_spec['tp'],
                        ds_spec['hs'],
                        ds_spec['gamma'],
                        ds_spec['duration'],
                        ds_spec['dt'],
                        ds_spec['this_seed'],
                        ds_spec['fft_min_duration'],
                        input_core_dims=[[],[],[],[],[],[],[]],
                        output_core_dims=[[],]*16,
                        vectorize=True,
                        dask='parallelized',
                        output_dtypes=['float',]*16
                        )
                        
# Assign the outputs to the xr.Dataset
for v, o in zip(output_variables.split(',')[:-1],outputs[:-1]): # ignore the returned seed - same as input
    ds_spec[v.strip()] = o
ds_spec

In [ ]:
# Use multiple processes to calculate timeseries
with dask.config.set(scheduler='processes'):
    with ProgressBar():
        ds_spec = ds_spec.compute()
ds_spec.to_netcdf(f'data/timeseries_stats/RAS_n{n_seed}_2Hz_fixedres.nc',mode='w')

# Analyse saved results

In [ ]:
ds_spec = xr.open_dataset(f'data/timeseries_stats/RAS_n{n_seed}_2Hz_fixedres.nc')
ds_spec['HmHs'] = ds_spec['Hmax']/ds_spec['Hs']
ds_std = ds_spec.std(dim='seed')
ds_mean = ds_spec.mean(dim='seed')
ds_max = ds_spec.max(dim='seed')
ds_CV = ds_std/ds_mean
ds_CV

In [ ]:
import hvplot.xarray
import holoviews as hv

In [ ]:
layout = []
for v in ds_mean.data_vars:
    layout.append(ds_mean[v].hvplot(x='duration',cmap='viridis',aspect=1,title=v))
layout = hv.Layout(layout).cols(3)

hv.save(layout,filename="figures/timeseries_stats/RAS_Mean_fixedres.html")

In [ ]:
layout = []
for v in ds_CV.data_vars:
    layout.append(ds_CV[v].hvplot(x='duration',cmap='viridis',aspect=1,title=v))
layout = hv.Layout(layout).cols(3)

hv.save(layout,filename="figures/timeseries_stats/RAS_CV_fixedres.html")

In [ ]:
layout = []
for v in ds_max.data_vars:
    layout.append(ds_max[v].hvplot(x='duration',cmap='viridis',aspect=1,title=v))
layout = hv.Layout(layout).cols(3)

hv.save(layout,filename="figures/timeseries_stats/RAS_Max_fixedres.html")

## Take a look at the largest wave event

In [ ]:
hv.extension('bokeh')

In [ ]:
ds_stacked = ds_spec.stack(sample=['tp','duration','gamma','seed'])
im=ds_stacked.Hmax.argmax()
largest_wave = ds_stacked.isel(sample=im)

display(largest_wave)

tp, duration, gamma,_ = np.array(largest_wave.sample.values.item())
seed = int(largest_wave.this_seed)
dt=float(ds_stacked['dt'])
hs=float(ds_stacked['hs'])
fft_duration=ds_stacked['fft_min_duration']
ts=time_domain_ras(tp,hs,gamma,duration,dt=dt,seed=seed,return_ts=True,fft_equiv_duration=fft_duration)
da_ts = xr.DataArray(ts,dims='time',coords={'time':np.arange(0,duration,dt)},name='timeseries')

# Double check that that the time series is generates correctly
from linearsim import wave_stats
print(f'ts statistics: {wave_stats(ts,1/dt)}')

da_ts.hvplot(x='time')